In [1]:
import pandas as pd
from datetime import datetime
import geopandas as gpd
from keplergl import KeplerGl
import pprint

In [2]:
# data exported from google spreadsheet to csv for the confirmed case sheet
df = pd.read_csv("time_series_2019-ncov - Confirmed.csv")

In [3]:
df.head()

,Province/State,Country/Region,First confirmed date in country (Est.),Lat,Long,1/21/2020 10:00 PM,1/22/2020 12:00 PM,1/23/2020 12:00 PM,1/24/2020 12:00 AM,1/24/2020 12:00 PM,...,1/28/2020 11:00 PM,1/29/2020 1:30 PM,1/29/2020 2:30 PM,1/29/2020 9:00 PM,1/30/2020 11:00 AM,1/31/2020 2:00 PM,1/31/2020 7:00 PM,2/1/2020 10:00 AM,2/2/2020 9:00 PM,2/3/2020 9:00 PM
0,Anhui,Mainland China,1/3/2020,31.82571,117.2264,NaN,1.0,9.0,15.0,15.0,...,152.0,152.0,152.0,200.0,200.0,237.0,237.0,297.0,408,480
1,Beijing,Mainland China,1/3/2020,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,...,91.0,111.0,111.0,111.0,114.0,139.0,139.0,168.0,191,212
2,Chongqing,Mainland China,1/3/2020,30.05718,107.8740,5.0,6.0,9.0,27.0,27.0,...,147.0,147.0,147.0,165.0,182.0,211.0,238.0,247.0,300,337
3,Fujian,Mainland China,1/3/2020,26.07783,117.9895,NaN,1.0,5.0,5.0,10.0,...,82.0,84.0,84.0,101.0,101.0,120.0,120.0,144.0,159,179
4,Gansu,Mainland China,1/3/2020,36.06110,103.8343,NaN,NaN,2.0,2.0,2.0,...,24.0,24.0,24.0,26.0,26.0,29.0,35.0,35.0,51,55


In [4]:
info_col = ["Province/State", "Country/Region", "First confirmed date in country (Est.)", "Lat", "Long"]

col_dayend = []
col_dayend_str = []
for col_str in df.columns:
    if not(col_str in info_col):
        col_obj = datetime.strptime(col_str, "%m/%d/%Y %I:%M %p") # parse 1/21/2020 10:00 PM to the format
        #print(col_obj.date())
        #print(col_obj)
        if col_obj.date() in col_dayend:
            col_dayend = col_dayend[:-1]
            col_dayend_str = col_dayend_str[:-1]
        col_dayend.append(col_obj.date())
        col_dayend_str.append(col_str)
        #print(col)
        #print(type(col))
#len(col_dayend)
col_dayend_str

['1/21/2020 10:00 PM',
 '1/22/2020 12:00 PM',
 '1/23/2020 12:00 PM',
 '1/24/2020 12:00 PM',
 '1/25/2020 10:00 PM',
 '1/26/2020 11:00 PM',
 '1/27/2020 8:30 PM',
 '1/28/2020 11:00 PM',
 '1/29/2020 9:00 PM',
 '1/30/2020 11:00 AM',
 '1/31/2020 7:00 PM',
 '2/1/2020 10:00 AM',
 '2/2/2020 9:00 PM',
 '2/3/2020 9:00 PM']

In [5]:
info_col2 = [ c for c in info_col if not c=="First confirmed date in country (Est.)" ]

In [6]:
df2 = df[info_col2+col_dayend_str]
#df2 = df2.set_index(info_col2)
df2.head()

,Province/State,Country/Region,Lat,Long,1/21/2020 10:00 PM,1/22/2020 12:00 PM,1/23/2020 12:00 PM,1/24/2020 12:00 PM,1/25/2020 10:00 PM,1/26/2020 11:00 PM,1/27/2020 8:30 PM,1/28/2020 11:00 PM,1/29/2020 9:00 PM,1/30/2020 11:00 AM,1/31/2020 7:00 PM,2/1/2020 10:00 AM,2/2/2020 9:00 PM,2/3/2020 9:00 PM
0,Anhui,Mainland China,31.82571,117.2264,NaN,1.0,9.0,15.0,60.0,70.0,106.0,152.0,200.0,200.0,237.0,297.0,408,480
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,36.0,51.0,68.0,80.0,91.0,111.0,114.0,139.0,168.0,191,212
2,Chongqing,Mainland China,30.05718,107.8740,5.0,6.0,9.0,27.0,75.0,110.0,132.0,147.0,165.0,182.0,238.0,247.0,300,337
3,Fujian,Mainland China,26.07783,117.9895,NaN,1.0,5.0,10.0,18.0,35.0,59.0,82.0,101.0,101.0,120.0,144.0,159,179
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,2.0,2.0,7.0,14.0,19.0,24.0,26.0,26.0,35.0,35.0,51,55


In [7]:
#df2.to_csv("time_series_2019-ncov-Confirmed-simplified.csv", index_label='ind')

In [8]:
df3 = pd.melt(df2, id_vars=info_col2, value_vars=col_dayend_str, var_name='datetime', value_name='confirmed_case')
df3.head()

,Province/State,Country/Region,Lat,Long,datetime,confirmed_case
0,Anhui,Mainland China,31.82571,117.2264,1/21/2020 10:00 PM,NaN
1,Beijing,Mainland China,40.18238,116.4142,1/21/2020 10:00 PM,10.0
2,Chongqing,Mainland China,30.05718,107.8740,1/21/2020 10:00 PM,5.0
3,Fujian,Mainland China,26.07783,117.9895,1/21/2020 10:00 PM,NaN
4,Gansu,Mainland China,36.06110,103.8343,1/21/2020 10:00 PM,NaN


In [9]:
df3 = df3.dropna(subset=["confirmed_case"])
df3.head()

,Province/State,Country/Region,Lat,Long,datetime,confirmed_case
1,Beijing,Mainland China,40.18238,116.4142,1/21/2020 10:00 PM,10.0
2,Chongqing,Mainland China,30.05718,107.8740,1/21/2020 10:00 PM,5.0
5,Guangdong,Mainland China,23.33841,113.4220,1/21/2020 10:00 PM,17.0
12,Hubei,Mainland China,30.97564,112.2707,1/21/2020 10:00 PM,270.0
13,Hunan,Mainland China,27.61041,111.7088,1/21/2020 10:00 PM,1.0


In [10]:
# this is to show the data using the time slider
dates = []
for d in df3.datetime.tolist():
    dt_obj = datetime.strptime(d, "%m/%d/%Y %I:%M %p") # parse 1/21/2020 10:00 PM to the format
    dates.append(dt_obj.strftime("%Y-%m-%d")+" 23:59")
df3['datetime'] = dates

In [11]:
df3.head()

,Province/State,Country/Region,Lat,Long,datetime,confirmed_case
1,Beijing,Mainland China,40.18238,116.4142,2020-01-21 23:59,10.0
2,Chongqing,Mainland China,30.05718,107.8740,2020-01-21 23:59,5.0
5,Guangdong,Mainland China,23.33841,113.4220,2020-01-21 23:59,17.0
12,Hubei,Mainland China,30.97564,112.2707,2020-01-21 23:59,270.0
13,Hunan,Mainland China,27.61041,111.7088,2020-01-21 23:59,1.0


In [12]:
#df3.to_csv("time_series_2019-ncov-Confirmed-simplified_long.csv", index_label='ind')

In [13]:
point_gdf = gpd.GeoDataFrame(df3, geometry=gpd.points_from_xy(df3["Long"], df3["Lat"]))

In [14]:
%run map_time_series.conf.py
# use the previous config as base
config

{'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': 'confirmed_case',
     'name': 'datetime',
     'type': 'timeRange',
     'value': [1579651140000, 1579737480000]}],
   'layers': [{'id': 'or7ukl4',
     'type': 'point',
     'config': {'dataId': 'confirmed_case',
      'label': 'confirmed_case',
      'color': [227, 26, 26],
      'columns': {'lat': 'Lat', 'lng': 'Long', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
 

In [15]:
map_1 = KeplerGl(height=900, config=config)
map_1.add_data(data=point_gdf, name="confirmed_case")
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'confirmed_case', 'name': 'da…

In [16]:
# add the time slider to map, need to manually add this to the config, because the save_to_html won't recognize the filter part

map_1.config["config"]['visState']["filters"] = [{"dataId": 'confirmed_case', "name": "datetime", 
                                                      'type': 'timeRange',
                                                      'value': [1579651140000, 1579737480000], ## a day
                                                      'name': 'datetime'}]


In [17]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(map_1.config)

{   'config': {   'mapState': {   'bearing': 0,
                                  'dragRotate': False,
                                  'isSplit': False,
                                  'latitude': 12.055250000000001,
                                  'longitude': 14.95205,
                                  'pitch': 0,
                                  'zoom': 2},
                  'mapStyle': {   'mapStyles': {},
                                  'styleType': 'dark',
                                  'threeDBuildingColor': [   9.665468314072013,
                                                             17.18305478057247,
                                                             31.1442867897876],
                                  'topLayerGroups': {},
                                  'visibleLayerGroups': {   '3d building': False,
                                                            'border': False,
                                                            'building

In [18]:
#if modification is done, uncomment the following and save the conf again
#with open("map_time_series.conf.py", "w") as f:
#    f.write('config = {}'.format(map_1.config))

In [19]:
map_1.save_to_html(file_name="map_time_series.html", config=config, read_only=True)

Map saved to map_time_series.html!
